<a href="https://colab.research.google.com/github/JonasZaoui/Segmentation-Pytorch/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module): #Creation de la classe DoubleConvo  (=2 fleches bleus) 
    def __init__(self, in_channels, out_channels): 
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False), #premiere convolution
            nn.BatchNorm2d(out_channels), #rajout, mais n'existait pas a l'epoque du paper
            nn.ReLU(inplace=True), # 1 ere relu
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False), #2e convolution
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True), #2e relu
        )

    def forward(self, x):
        return self.conv(x) #on applique à x la self.conv = double convolution

class UNET(nn.Module):
    def __init__( #on ecrit les elements qui nous serviront pour le forward 
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList() #liste des couches montantes 
        self.downs = nn.ModuleList() #liste des couches descendantes 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #pooling appliqué 

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature)) #on rajoute a la liste descendante une double convolution ; 
            in_channels = feature # l'input prochaine est l'output d'avant

        # Up part of UNET
        for feature in reversed(features): 
            self.ups.append(
                nn.ConvTranspose2d( #fleche verte de l'architecture
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2) #derniere couche en bas
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1) #fleche turquoise, conv 1x1

    def forward(self, x): #x = feature de depart 
        skip_connections = []

        for down in self.downs: #down = DoubleConv(in_channels,feature)
            x = down(x) #j'applique une doubleconv sur l'output de la doubleconv d'avant. 
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x) #derniere ligne : le goulot. 
        skip_connections = skip_connections[::-1] #on renverse, car on a besoin des features a l'envers 

        for idx in range(0, len(self.ups), 2): #par pas de 2 car dans la liste self.ups=[transposeconv, doubleconv, transposeconv, doubleconv....]
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2] #on recupere la skip connection de la partie descendante

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:]) 

            concat_skip = torch.cat((skip_connection, x), dim=1) #on les concatene
            x = self.ups[idx+1](concat_skip) #on fait une double conv

        return self.final_conv(x)

def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    assert preds.shape == x.shape

if __name__ == "__main__":
    test()

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir #dir du train ou du test set
        self.mask_dir = mask_dir 
        self.transform = transform
        self.images = os.listdir(image_dir) #liste des images 

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask


